<h1><center>ASSIGNMENT 3</center></h1>
## TASK 1: Ranking by desirability
Below, we load the data from the new file provided by Zillow. The interesting variables are found out like Assignment 2 and made into a dataframe for further processing.

In [35]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

df = pd.read_csv("properties_2017.csv")
col_array=list(df)
data_array=(len(df.index)-df.isnull().sum())/len(df.index)
col_selected=[]

for i in range(0,len(data_array)):
    if data_array[i]>=0.5:
        col_selected.append(col_array[i])

remove_labels=['fips','propertyzoningdesc','rawcensustractandblock','censustractandblock','assessmentyear','taxdelinquencyyear']

for x in remove_labels:
    if x in col_selected:
        col_selected.remove(x)
    
selected_df=df[col_selected].copy()

After selecting the interesting columns, we further narrow down out choice. We select a number of columns which are most likely to determine the price of a house and form a scoring function using them. We then score each of the house and find out the 10 houses which score the most and least according to our function.

In [34]:
import math

scoring_cols=['parcelid','bathroomcnt','buildingqualitytypeid','finishedsquarefeet12','unitcnt','yearbuilt','taxamount']
temp_df=selected_df[scoring_cols].copy()
temp_df=temp_df.fillna(temp_df.mean())

normalized_df=pd.DataFrame()

for col in scoring_cols:
    if col=='parcelid':
        normalized_df[col]=temp_df[col]
    else:
        normalized_df[col]=(temp_df[col] - temp_df[col].mean()) / (temp_df[col].max() - temp_df[col].min())

def scoring_fn(bath,quality,area,unit,year,tax):
    return (0.05*bath)+(0.3*(1-quality))+(0.3*area)+(0.1*unit)+(0.1*year)+(0.15*tax)
    
normalized_df['score'] = normalized_df.apply(lambda row: scoring_fn(row['bathroomcnt'], row['buildingqualitytypeid'],row['finishedsquarefeet12'],row['unitcnt'],row['yearbuilt'],row['taxamount']), axis=1)

normalized_df=normalized_df.sort_values('score')
print('Best 10 houses:\n')
print(normalized_df.head(10)[['parcelid']].to_string(index=False))
print('\nWorst 10 houses:\n')
print(normalized_df.tail(10)[['parcelid']].to_string(index=False))

Best 10 houses:

parcelid
12899194
12876709
11775769
11772640
13017031
13017006
12120679
11771438
12088683
11797476

Worst 10 houses:

parcelid
 11642356
 11681241
 10738101
 12969862
 12146164
 10868753
 11887283
 11857690
167638594
 10944745


### Description of cost function
The columns making up the cost function has been selected according to their potential to play an important factor in determining the pricing of a house. In doing that, the columns had been normalized because each column has a different unit and largely varying range. The weights given to the different columns has been assigned so as to reflect the effect of that factor on the price.
<p>The columns selected for the weight function are the following:</p>
<p>
<strong>Building quality type: </strong>The quality of a building would be very important in determining the price of the building. The higher the quality, the greater would be the price of it. A lower number in the quality column denotes a better building. So the scoring function has been taken to subtract the normalized quality from 1 so that a lower value may reflect highly on the score.
</p>
<p>
<strong>Finished square feet: </strong>Higher the area of the house, the greater will be the price. The importance of the finished square feet in determining the price of a house is reflected in its weight.
</p>
<p>
<strong>Unit count: </strong>A house having more units would have a higher price tag than one with smaller number of units.
</p>
<p>
<strong>Year built: </strong>A house built more recently is more likely to have a higher price than old ones. In assignment 2, we saw that the quality of house is related to the year they were built in.
</p>
<p>
<strong>Tax on property: </strong>Highly taxed houses are also the more expensive ones.
</p>
<p>
<strong>Bathroom count: </strong>Not a very important as seen from the weight. However a house having more bathrooms would also be roomier and so more expensive.
</p>
<p>A large number of factors play into the price of a house. However the factors chosen here are the most likely to affect the price. Their weight has been arranged so that they portray an accurate picture of the relative importance of the different factors involved. As such, the cost function should be very accurate.</p>

## TASK 2: Pairwise distance function

In [ ]:
import math

factor_cols=['parcelid','latitude','longitude','buildingqualitytypeid','finishedsquarefeet12','unitcnt','taxamount']
temp_df=selected_df[factor_cols].copy()
temp_df=temp_df.fillna(temp_df.mean())

normalized_df=pd.DataFrame()

for col in factor_cols:
    if col=='parcelid':
        normalized_df[col]=temp_df[col]
    else:
        normalized_df[col]=(temp_df[col] - temp_df[col].mean()) / (temp_df[col].max() - temp_df[col].min())

def distance_fn(lat1,lat2,long1,long2,quality1,quality2,area1,area2,unit1,unit2,tax1,tax2):
    lat_diff=math.pow(math.fabs(lat1-lat2),2)
    long_diff=math.pow(math.fabs(long1-long2),2)
    qual_diff=math.pow(math.fabs(quality1-quality2),2)
    area_diff=math.pow(math.fabs(area1-area2),2)
    unit_diff=math.pow(math.fabs(unit1-unit2),2)
    tax_diff=math.pow(math.fabs(tax1-tax2),2)
    return math.sqrt(lat_diff+long_diff+qual_diff+area_diff+unit_diff+tax_diff)
    
normalized_df['score'] = normalized_df.apply(lambda row: scoring_fn(row['bathroomcnt'], row['buildingqualitytypeid'],row['finishedsquarefeet12'],row['unitcnt'],row['yearbuilt'],row['taxamount']), axis=1)

normalized_df=normalized_df.sort_values('score')
print('Best 10 houses:\n')
print(normalized_df.head(10)[['parcelid']].to_string(index=False))
print('\nWorst 10 houses:\n')
print(normalized_df.tail(10)[['parcelid']].to_string(index=False))